# Шаг 1

Необходимо скачать CSV-файл - «transactions_for_dz2.csv», создать таблицу transaction_bd со всеми полями, загрузить данные из файла в таблицу.
Также необходимо скачать второй CSV-файл - «client_info.csv», создать таблицу customer_bd со всеми полями, загрузить данные из файла в таблицу 

In [1]:
import sqlite3
import pandas as pd
import csv

## Проверяем данные

In [2]:
path_1 = "transactions_for_dz2.csv"
path_2 = "client_info.csv"

try:
    df_1 = pd.read_csv(path_1)
    df_2 = pd.read_csv(path_2, delimiter=';')
except:
    url_1 = 'https://drive.google.com/file/d/1GsKxPQ3cp1_uuyoLoAQLlsHTLEbXOMIo/view?usp=share_link'
    url_2 = 'https://drive.google.com/file/d/1oFzZfwHBtP0NOulBYlp4osIY5NIPIdn-/view?usp=share_link'

    # выделяем хвост адреса
    file_id_1 = url_1.split('/')[-2]
    file_id_2 = url_2.split('/')[-2]

    # заменяем основную часть адреса (одинаковая для всех гугл-дисков) и прибавляем хвост
    read_url_1='https://drive.google.com/uc?id=' + file_id_1
    read_url_2='https://drive.google.com/uc?id=' + file_id_2

    df_1 = pd.read_csv(read_url_1)
    df_2 = pd.read_csv(read_url_2, delimiter=';')

display(df_1)
display(df_2)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66
...,...,...,...,...,...
1048570,1048570,2023-04-20 10:07:13,2380,3780,325.64
1048571,1048571,2023-04-20 10:07:28,738,5151,20.38
1048572,1048572,2023-04-20 10:07:33,1000,5417,182.79
1048573,1048573,2023-04-20 10:07:39,3028,6439,455.44


,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767
...,...,...,...,...,...
4983,2023-04-20,2999-12-31,Havilah,2000,4737
4984,2023-04-20,2999-12-31,Hazelyn,2000,2301
4985,2023-04-20,2999-12-31,Helaina,2000,3238
4986,2023-04-20,2999-12-31,Helene,2000,3946


## Чистим данные

In [2]:
df_1.duplicated().sum()

0

In [5]:
df_1.isna().sum()

TRANSACTION_ID    0
TX_DATETIME       0
CUSTOMER_ID       0
TERMINAL_ID       0
TX_AMOUNT         0
dtype: int64

In [9]:
df_2.duplicated().sum()

0

In [10]:
df_2.isna().sum()

START_DT       0
END_DT         0
CLIENT_NAME    0
YEAR_BIRTH     0
CLIENT_ID      0
dtype: int64

In [ ]:
# Данные в обеих таблицах не имеют пропусков и дубликатов, их можно заносить в базы sql

## Создаем общую базу и заливаем данные в 2 таблицы

In [3]:
con = sqlite3.connect('transaction_bd.db')   # создается или открывается БД
cur = con.cursor()
cur.execute("DROP TABLE IF EXISTS transaction_bd;")
cur.execute("CREATE TABLE transaction_bd (TRANSACTION_ID INTEGER PRIMARY KEY AUTOINCREMENT, TX_DATETIME NUMERIC, \
            CUSTOMER_ID INTEGER, TERMINAL_ID INTEGER, TX_AMOUNT REAL);")

with open('F:/МФТИ\Системы хранения и обработки/HW 3 БД с джойнами/transactions_for_dz2.csv','r') as fin:
    # csv.DictReader по умолчанию использует первую строку под заголовки столбцов
    dr = csv.DictReader(fin, delimiter=",")
    to_db = [(i['TRANSACTION_ID'], i['TX_DATETIME'], i['CUSTOMER_ID'], i['TERMINAL_ID'], i['TX_AMOUNT']) for i in dr]

cur.executemany("INSERT INTO transaction_bd (TRANSACTION_ID, TX_DATETIME, CUSTOMER_ID, TERMINAL_ID, TX_AMOUNT) VALUES (?, ?, ?, ?, ?);", to_db)

cur.execute("DROP TABLE IF EXISTS customer_bd;")
cur.execute("CREATE TABLE customer_bd (CLIENT_ID INTEGER PRIMARY KEY AUTOINCREMENT, START_DT NUMERIC, \
            END_DT NUMERIC, CLIENT_NAME TEXT, YEAR_BIRTH TEXT);")

with open('F:/МФТИ/Системы хранения и обработки/HW 3 БД с джойнами/client_info.csv','r') as fin:
    # csv.DictReader по умолчанию использует первую строку под заголовки столбцов
    dr1 = csv.DictReader(fin, delimiter=";")
    to_db_1 = [(i['START_DT'], i['END_DT'], i['CLIENT_NAME'], i['YEAR_BIRTH'], i['CLIENT_ID']) for i in dr1]

cur.executemany("INSERT INTO customer_bd (START_DT, END_DT, CLIENT_NAME, YEAR_BIRTH, CLIENT_ID) VALUES (?, ?, ?, ?, ?);", to_db_1)

con.commit()

In [4]:
# ф-я выполнения скрипта и вывода результата

def sql_fetch(con, sql_script):
    cur = con.cursor()
    cur.execute(sql_script)
    rows = cur.fetchall()
    for row in rows:
        print(row)

## Проверяем структуру таблиц

Необходимая структура таблицы transaction_bd:

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 0) - id транзакции
(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - дата транзакции
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - id клиента
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0) - id терминала
(4, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакции

Необходимая структура таблицы customer_bd:

(0, 'CLIENT_ID', 'INTEGER', 0, None, 0) - id клиента
(1, 'START_DT', 'NUMERIC', 0, None, 0) - дата начало записи о клиенте
(2, 'END_DT', 'NUMERIC', 0, None, 0) - дата закрытия записи о клиенте
(3, 'CLIENT_NAME', 'TEXT', 0, None, 0) - название клиента
(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0) - дата рождение клиента

In [10]:
sql_fetch(con, "pragma table_info(transaction_bd);")
sql_fetch(con, "pragma table_info(customer_bd);")

(0, 'TRANSACTION_ID', 'INTEGER', 0, None, 1)
(1, 'TX_DATETIME', 'NUMERIC', 0, None, 0)
(2, 'CUSTOMER_ID', 'INTEGER', 0, None, 0)
(3, 'TERMINAL_ID', 'INTEGER', 0, None, 0)
(4, 'TX_AMOUNT', 'REAL', 0, None, 0)
(0, 'CLIENT_ID', 'INTEGER', 0, None, 1)
(1, 'START_DT', 'NUMERIC', 0, None, 0)
(2, 'END_DT', 'NUMERIC', 0, None, 0)
(3, 'CLIENT_NAME', 'TEXT', 0, None, 0)
(4, 'YEAR_BIRTH', 'TEXT', 0, None, 0)


Структура таблиц отличается от заданной, так как имеются первичные ключи (последний параметр равен 1 у transaction_id и client_id.

In [6]:
pd.read_sql('''
    SELECT
        count(*)
    FROM customer_bd C
    left join transaction_bd T ON T.CUSTOMER_ID = C.CLIENT_ID
    WHERE T.TX_DATETIME > "2023-04-20"
    ORDER BY C.CLIENT_ID DESC
       ''', con)

,count(*)
0,3386


In [7]:
pd.read_sql('''
    SELECT
        count(*)
    FROM customer_bd C
    left join transaction_bd T ON T.CUSTOMER_ID = C.CLIENT_ID
    AND T.TX_DATETIME > "2023-04-20"
    ORDER BY C.CLIENT_ID DESC
       ''', con)

,count(*)
0,6099


Это проверка для итогового теста

# Шаг 2

## a. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня, относительно которого мы считаем).

Поскольку нам нужен разовый запрос о датах, создадим табличное выражение (CTE) и с его помощью разнесем 2 условия по датам и агрегирующие функции. Для вычисления интервала с учетом всех дней, часов и минут нужна функция julianday.

In [281]:
pd.read_sql(''' with cte as
    (SELECT
        T.CUSTOMER_ID, 
        C.CLIENT_NAME, 
        T.TX_DATETIME,
        LAG(T.TX_DATETIME) OVER w as LAG_DATETIME,
        julianday(T.TX_DATETIME) - julianday(LAG(T.TX_DATETIME) OVER w) as LAG_DAYS,
        C.END_DT
    FROM transaction_bd T
    LEFT JOIN customer_bd C ON T.CUSTOMER_ID = C.CLIENT_ID
    WHERE C.END_DT >= '2023-05-01' 
    window w as (PARTITION BY T.CUSTOMER_ID ORDER BY T.TX_DATETIME)
    )
    SELECT DISTINCT
        cte.CUSTOMER_ID,
        cte.CLIENT_NAME, 
        cte.TX_DATETIME,
        cte.LAG_DATETIME,
        cte.LAG_DAYS,
        cte.END_DT
    FROM cte    
    WHERE cte.LAG_DAYS > 35

    ''', con)

,CUSTOMER_ID,CLIENT_NAME,TX_DATETIME,LAG_DATETIME,LAG_DAYS,END_DT
0,24,Kiarra,2023-02-13 13:51:25,2023-01-04 10:14:34,40.150590,2999-12-31
1,480,Vaishnavi,2023-04-15 16:52:55,2023-02-20 07:59:43,54.370278,2999-12-31
2,639,Malayna,2023-03-29 18:25:10,2023-02-22 10:42:08,35.321551,2999-12-31
3,707,Becca,2023-03-06 07:59:25,2023-01-30 05:12:10,35.116146,2999-12-31
4,812,Avionna,2023-04-14 16:22:54,2023-01-21 16:25:44,82.998032,2999-12-31
5,896,Macarena,2023-03-17 18:32:43,2023-01-26 06:55:31,50.484167,2999-12-31
6,1129,Kavya,2023-04-15 13:22:09,2023-03-04 17:08:35,41.842755,2999-12-31
7,1299,Nalah,2023-03-24 03:12:47,2023-01-17 20:39:48,65.272905,2999-12-31
8,1334,Blakley,2023-04-05 10:37:57,2023-02-08 15:37:39,55.791875,2999-12-31
9,1459,Nalia,2023-03-16 06:11:02,2023-01-30 08:30:00,44.903495,2999-12-31


Имеется дублирующая строка по клиенту Willamina, у которой 2 раза был перерыв дольше 35 дней между транзакциями. Видимо, select distinct не работает, если есть дополнительные строки в запросе, но если задать для вывода только CUSTOMER_ID, то повторов не будет.

In [282]:
pd.read_sql(''' with cte as
    (SELECT
        T.CUSTOMER_ID, 
        C.CLIENT_NAME, 
        T.TX_DATETIME,
        LAG(T.TX_DATETIME) OVER w as LAG_DATETIME,
        julianday(T.TX_DATETIME) - julianday(LAG(T.TX_DATETIME) OVER w) as LAG_DAYS,
        C.END_DT
    FROM transaction_bd T
    LEFT JOIN customer_bd C ON T.CUSTOMER_ID = C.CLIENT_ID
    WHERE C.END_DT >= '2023-05-01' 
    window w as (PARTITION BY T.CUSTOMER_ID ORDER BY T.TX_DATETIME)
    )
    SELECT DISTINCT
        cte.CUSTOMER_ID
    FROM cte    
    WHERE cte.LAG_DAYS > 35

    ''', con)

,CUSTOMER_ID
0,24
1,480
2,639
3,707
4,812
5,896
6,1129
7,1299
8,1334
9,1459


## b. Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня, относительно которого мы считаем).

In [132]:
pd.read_sql('''
    SELECT 
        T.CUSTOMER_ID, 
        max(T.TX_AMOUNT) as MAX_TX_AMOUNT,
        min(T.TX_AMOUNT) as MIN_TX_AMOUNT,
        C.END_DT
    FROM transaction_bd T
    JOIN customer_bd C ON T.CUSTOMER_ID = C.CLIENT_ID
    WHERE C.END_DT >= '2023-05-01' 
    GROUP BY 
        1
    HAVING max(T.TX_AMOUNT)>50000    
    ''', con)

,CUSTOMER_ID,MAX_TX_AMOUNT,MIN_TX_AMOUNT,END_DT
0,3013,77212.50,1.46,2023-09-29
1,3494,53213.00,85.76,2999-12-31
2,4253,51937.25,37.95,2999-12-31


Поскольку запрос "клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня, относительно которого мы считаем" повторяется в каждом задании, сделаем соответствующее представление (view). Сделаем аналогичный запрос с view.

In [217]:
cur.execute('''DROP VIEW IF EXISTS V''')
con.commit()

cur.execute('''CREATE view V as
SELECT *
FROM customer_bd C 
    JOIN transaction_bd T ON C.CLIENT_ID = T.CUSTOMER_ID
    WHERE C.END_DT >= '2023-05-01';
    ''')
con.commit()

In [218]:
pd.read_sql('''
    SELECT 
        V.CUSTOMER_ID, 
        max(V.TX_AMOUNT) as MAX_TX_AMOUNT,
        min(V.TX_AMOUNT) as MIN_TX_AMOUNT,
        V.END_DT
    FROM V 
    GROUP BY 
        1
    HAVING MAX_TX_AMOUNT>50000    
    ''', con)

,CUSTOMER_ID,MAX_TX_AMOUNT,MIN_TX_AMOUNT,END_DT
0,3013,77212.50,1.46,2023-09-29
1,3494,53213.00,85.76,2999-12-31
2,4253,51937.25,37.95,2999-12-31


Результаты аналогичны.

## c. Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций. Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня, относительно которого мы считаем).

Поскольку дано 2 условия (дата закрытия записи и сумма дневных vs. сумма вечерних транзакций), необходимо сделать табличное выражение cte, чтобы разнести эти условия и агрегирующие функции по разным запросам.

In [259]:
pd.read_sql('''
        with cte as (
        SELECT C.CLIENT_ID,
            CASE WHEN strftime('%H', T.TX_DATETIME) < '12:00:00' THEN sum(T.TX_AMOUNT) ELSE 0 END AS TOTAL_DAY_TRANSACTIONS,
            CASE WHEN strftime('%H', T.TX_DATETIME) >= '12:00:00' THEN sum(T.TX_AMOUNT) ELSE 0 END AS TOTAL_NIGHT_TRANSACTIONS,
            C.END_DT
        FROM customer_bd C 
        JOIN transaction_bd T ON T.CUSTOMER_ID = C.CLIENT_ID
        WHERE C.END_DT >= '2023-05-01'
        GROUP BY 1
        )
        SELECT
            cte.CLIENT_ID,
            cte.TOTAL_DAY_TRANSACTIONS,
            cte.TOTAL_NIGHT_TRANSACTIONS,
            cte.END_DT
        FROM cte
        WHERE cte.TOTAL_DAY_TRANSACTIONS > cte.TOTAL_NIGHT_TRANSACTIONS
        ORDER BY 1 DESC
        LIMIT 10;

        ''', con)

,CLIENT_ID,TOTAL_DAY_TRANSACTIONS,TOTAL_NIGHT_TRANSACTIONS,END_DT
0,4997,131288.62,0,2999-12-31
1,4996,27135.67,0,2999-12-31
2,4993,84892.64,0,2999-12-31
3,4992,118447.24,0,2999-12-31
4,4988,176279.64,0,2999-12-31
5,4982,85427.60,0,2999-12-31
6,4978,6211.64,0,2999-12-31
7,4976,133362.85,0,2999-12-31
8,4975,12438.37,0,2999-12-31
9,4973,130284.26,0,2999-12-31


## d. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня, относительно которого мы считаем).

In [210]:
pd.read_sql('''
    SELECT 
        T.CUSTOMER_ID, 
        C.CLIENT_NAME,
        T.TX_DATETIME, 
        max(T.TX_AMOUNT) as MAX_TX_AMOUNT,
        C.END_DT
    FROM customer_bd C
    LEFT JOIN transaction_bd T ON C.CLIENT_ID = T.CUSTOMER_ID
    WHERE C.END_DT >= '2023-05-01'
    GROUP BY 
        1
    ''', con)

,CUSTOMER_ID,CLIENT_NAME,TX_DATETIME,MAX_TX_AMOUNT,END_DT
0,NaN,Sofia,None,NaN,2999-12-31
1,0.0,Eleni,2023-02-01 05:30:38,1258.86,2999-12-31
2,1.0,Jocelyn,2023-02-07 18:42:49,17152.00,2999-12-31
3,2.0,Bria,2023-02-21 20:09:34,1720.11,2999-12-31
4,5.0,Zimal,2023-03-21 13:00:37,1156.11,2999-12-31
...,...,...,...,...,...
3732,4991.0,Tyanna,2023-01-25 15:29:38,1070.26,2999-12-31
3733,4992.0,Logan,2023-03-10 19:51:24,839.03,2999-12-31
3734,4993.0,Oakland,2023-01-13 15:50:46,1054.57,2999-12-31
3735,4996.0,Royal,2023-01-11 09:47:27,224.62,2999-12-31


## e. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме (не учитываем, что клиент может быть недействующим)

Поскольку группировать оконную функцию CUME_DIST в данном случае надо по агрегирующему показателю (сумме транзакций по дням), необходимо сделать отдельную таблицу (cte).

In [216]:
pd.read_sql('''with cte as(
SELECT
    date(TX_DATETIME) as TX_DATE,
    sum(TX_AMOUNT) as TX_SUM
    FROM transaction_bd
    GROUP BY 
    1
)
SELECT 
        cte.TX_DATE,
        cte.TX_SUM,
        CUME_DIST() OVER (ORDER BY cte.TX_SUM) as CUME_DIST
    FROM cte
    ORDER BY 
    1
''', con)

,TX_DATE,TX_SUM,CUME_DIST
0,2023-01-01,4827656.26,0.018182
1,2023-01-02,4862551.41,0.027273
2,2023-01-03,5058973.71,0.209091
3,2023-01-04,4938142.47,0.036364
4,2023-01-05,5002954.23,0.109091
...,...,...,...
105,2023-04-16,5299386.81,0.909091
106,2023-04-17,5100973.31,0.318182
107,2023-04-18,5233557.39,0.754545
108,2023-04-19,5194846.44,0.636364


In [ ]:
con.close()